### Process Tiff Stack

In [5]:
from pvd_par import PVD
import os

In [6]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np

In [7]:
def create_skeleton_plot(timepoints=4):
    # Timepoint of interest (0 to 3)
    for timepoint in range(timepoints):
        # Prepare skeleton data for plotting
        image_stack = np.transpose(pvd.skeletonized_data[timepoint], (1, 2, 0))
        x, y, z = image_stack.shape
        Y, X, Z = np.meshgrid(np.arange(y), np.arange(x), np.arange(z))
        colors = image_stack.ravel()

        # Omit background (0-valued) voxels
        visible_mask = colors != 0

        # Extract coordinates for skeleton tips
        tips_z = [point[0] for point in pvd.tips[timepoint]]
        tips_x = [point[1] for point in pvd.tips[timepoint]]
        tips_y = [point[2] for point in pvd.tips[timepoint]]

        # Extract coordinates for skeleton knots
        knot_z = [point[0] for point in pvd.knots[timepoint]]
        knot_x = [point[1] for point in pvd.knots[timepoint]]
        knot_y = [point[2] for point in pvd.knots[timepoint]]

        # Plotly figure
        fig = go.Figure()

        # Skeleton tips
        fig.add_trace(go.Scatter3d(
            x=tips_x,
            y=tips_y,
            z=tips_z,
            mode='markers',  # Use 'lines' for a line plot or 'markers+lines' for both
            marker=dict(
                size=5,
                color='red',  # You can customize the color
                opacity=.9
            )
        ))

        # Skeleton knots
        fig.add_trace(go.Scatter3d(
            x=knot_x,
            y=knot_y,
            z=knot_z,
            mode='markers',  # Use 'lines' for a line plot or 'markers+lines' for both
            marker=dict(
                size=4,
                color='blue',  # You can customize the color
                opacity=.9
            )
        ))

        # Skeleton structure
        fig.add_trace(go.Scatter3d(
            x=X.ravel()[visible_mask],
            y=Y.ravel()[visible_mask],
            z=Z.ravel()[visible_mask],
            mode='markers',
            marker=dict(
                size=2,
                color='black',
                colorscale='Viridis',
                opacity=.1
            )
        ))

        # Update layout
        fig.update_layout(
            title=f"C. Elegans PVD Neuron: <b>{session}</b><br>Timepoint: <b>{timepoint}</b>",
            scene=dict(
                xaxis_title='X (pixels)',
                yaxis_title='Y (pixels)',
                zaxis_title='Z (image index)',
                aspectmode='manual',
                aspectratio=dict(x=1, y=1, z=.2),  # Adjust z-axis scale if desired
                zaxis=dict(range=[0, pvd.skeletonized_data[timepoint].shape[0]]),  # Set z-axis bounds
                xaxis=dict(range=[0, pvd.skeletonized_data[timepoint].shape[1]]),  # Set x-axis bounds
                yaxis=dict(range=[0, pvd.skeletonized_data[timepoint].shape[2]]),  # Set y-axis bounds

            ),
            autosize=True

        )

        # Save the plot to an HTML file
        pio.write_html(fig, file=f'plots/{session}_skeleton_{timepoint}.html', auto_open=False)

In [8]:
# Function to get files and paths
def scan_directories(data_directory):
    dataset_list = []
    session_list = []
    file_list = []

    for root, dirs, files in os.walk(data_directory):
        # Check if the current path contains "exclude" or "movement"
        if "exclude" in root.lower() or "movement" in root.lower():
            continue  # Skip this directory

        for file in files:
            if file.endswith('.tif'):
                path_parts = os.path.normpath(root).split(os.sep)
                if len(path_parts) >= 3:
                    dataset = path_parts[-2]
                    session = path_parts[-1]
                    dataset_list.append(dataset)
                    session_list.append(session)
                    file_list.append(file)

    return dataset_list, session_list, file_list

In [9]:
# Get all tiff stacks and their paths
data_dir = 'S:/pvd_data/'
datasets, sessions, files = scan_directories(data_dir)
print(f"Located {len(files)} tiff stacks")

Located 55 tiff stacks


In [10]:
# Execute pipeline
for ii, file in enumerate(files):
    # Set paths
    data_path = 'S:/pvd_data'
    dataset = datasets[ii]
    session = sessions[ii]
    tiff_stack_path = f"{data_path}/{dataset}/{session}/{file}"
    output_path = f"{data_path}/{dataset}/{session}/"

    # Process stack
    print(f"Processing tiff stack {ii}/{len(files)}...")
    pvd = PVD(tiff_stack_path)
    pvd.run_pipeline()
    
    # Save relevant numpy arrays
    print(f"Saving numpy arrays to disk...")
    pvd.save_results(output_path, save_tiff=False)
    # Save plots
    print(f"Creating skeleton visualizations...")
    create_skeleton_plot()

Processing tiff stack 0/55...
Starting pipeline
Loaded raw data with shape: (4, 92, 2044, 2042)
Cropped raw data to shape: (4, 92, 2000, 2000)
Skeletonizing timepoint 0Skeletonizing timepoint 1

Skeletonizing timepoint 2
Skeletonizing timepoint 3
Completed skeletonizing timepoint 3
Completed skeletonizing timepoint 0
Completed skeletonizing timepoint 2
Completed skeletonizing timepoint 1
Skeletonized data shape: [(92, 2000, 2000), (92, 2000, 2000), (92, 2000, 2000), (92, 2000, 2000)]
Number of tips per timepoint: [214, 186, 136, 98]
Number of knots per timepoint: [475, 452, 311, 246]
Pipeline completed
Processing tiff stack 1/55...
Starting pipeline
Loaded raw data with shape: (4, 92, 2044, 2042)
Cropped raw data to shape: (4, 92, 2000, 2000)
